In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Collect video files

In [ ]:
import os

# Path to the main folder containing the subfolders
main_folder_path = '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos'
file_names=[]
# Loop through each subfolder and access the files
for subfolder_name in os.listdir(main_folder_path):
    subfolder_path = os.path.join(main_folder_path, subfolder_name)

    # Check if the current item is a subfolder
    if os.path.isdir(subfolder_path):
        # Loop through all files in the subfolder
        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)
            file_names.append(file_path)
print(file_names)

['/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_abortion.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_ability.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_Abraham.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_absent.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_accept.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_access.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_account.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_accuse.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_act.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/A/gsl_add.mp4', '/content/drive/MyDrive/signlanguage_videos/data_collection/

In [ ]:
import os

# Path to the main folder containing the subfolders
main_folder_path = '/content/drive/MyDrive/signlanguage_videos/Trimmed_Augmented_Videos'
file_names_aug=[]

for file_name in os.listdir(main_folder_path):
    file_path = os.path.join(main_folder_path, file_name)
    file_names_aug.append(file_path)
print(len(file_names_aug))

9750


In [ ]:
print(len(file_names))

984


In [ ]:
remaining = []
for f in file_names:
  parts = f.split('/')
  nm=parts[-1]
  flag=0
  for fl in file_names_aug:
    parts_fl=fl.split('/')
    nm_fl=parts_fl[-1]
    ind=nm_fl.find('gsl')
    nm_fl_gsl=nm_fl[ind:]
    if nm == nm_fl_gsl:
      flag=1
      break
  if flag==0:
    remaining.append(f)
print(len(remaining))
print(remaining)

9
['/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/J/.DS_Store', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/L/.DS_Store', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/N/.DS_Store', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/M/.DS_Store', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/O/.DS_Store', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/P/.DS_Store', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/Q/.DS_Store', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/Q/T', '/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/S/gsl_sheep.mp4']


Augmentation Library for techniques: ['Multiply', 'HorizontalFlip', 'InvertColor', 'PiecewiseAffineTransform', 'RandomTranslate', 'RandomRotate60', 'VerticalFlip', 'GaussianBlur', 'RandomRotate30']

In [ ]:
!pip install git+https://github.com/okankop/vidaug

  Cloning https://github.com/okankop/vidaug to /tmp/pip-req-build-cztz50b7
  Running command git clone --filter=blob:none --quiet https://github.com/okankop/vidaug /tmp/pip-req-build-cztz50b7
  Resolved https://github.com/okankop/vidaug to commit 1c1ddf2640fe4a9171267d64ae5e3bd70c24d54a
  Preparing metadata (setup.py) ... done
  Created wheel for vidaug: filename=vidaug-0.1-py3-none-any.whl size=13019 sha256=4721211952243211c238f27862f28b9229527f8c7a1ed690eeb5aa4e819bd9dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-ctrk0s4q/wheels/ea/46/d3/af9760dd8b5d9138de88563543ca9d821dabb66cf9081528a9
Successfully built vidaug


Code for Video Camera Perspective Change

In [ ]:
import cv2
import numpy as np

def change_perspective(image, src_points, dst_points):
    # Define the transformation matrix
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    # Apply the perspective transformation
    transformed_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))
    return transformed_image

def update_frame(image, height_reduction_percentage=20, width_reduction_percentage=30):
  src_points = np.float32([[0, 0], [image.shape[1] - 1, 0], [0, image.shape[0] - 1], [image.shape[1] - 1, image.shape[0] - 1]])
  reduction_pixels_h = int(image.shape[0] * (height_reduction_percentage / 100))
  reduction_pixels_w = int(image.shape[1] * (width_reduction_percentage / 100))
  # Adjust destination points for the desired transformation
  dst_points = np.float32([[reduction_pixels_w, 0], [image.shape[1] - 1, reduction_pixels_h], [reduction_pixels_w, image.shape[0] - 1], [image.shape[1] - 1, image.shape[0] - 1 - reduction_pixels_h]])
  # Change the perspective of the image
  transformed_image = change_perspective(image, src_points, dst_points)
  return transformed_image


Augmentation Script

In [ ]:
import os
import cv2
import numpy as np
import vidaug.augmentors as va

output_dir = '/content/drive/MyDrive/signlanguage_videos/Trimmed_Augmented_Videos'
flag=0
i=0
# Loop over each video file in the input directory
for video_path in file_names:
    # if video_path!='/content/drive/MyDrive/signlanguage_videos/data_collection/Trimmed videos/S/gsl_sheep.mp4':
    #   i+=1
    #   continue
    # Load the video file
    cap = cv2.VideoCapture(video_path)
    file=video_path.split('/')[-1]

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = (width, height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # collect all frames of the video
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frames.append(frame)
        else:
            break
    cap.release()

    # Using VideoAug library
    seq1 = va.Sequential([va.Multiply(3)])
    seq2 = va.Sequential([va.HorizontalFlip()])
    seq3 = va.Sequential([va.InvertColor()])
    seq4 = va.Sequential([va.PiecewiseAffineTransform(displacement=3, displacement_kernel=3, displacement_magnification=3)])
    seq5 = va.Sequential([va.RandomTranslate(300, 250)])
    #seq6 = va.Sequential([va.RandomResize()])
    seq7 = va.Sequential([va.RandomRotate(60)])
    seq8 = va.Sequential([va.VerticalFlip()])
    seq9 = va.Sequential([va.GaussianBlur(2)])
    seq10 = va.Sequential([va.RandomRotate(30)])

    seqs=[seq1, seq2, seq3, seq4, seq5, seq7, seq8, seq9, seq10]
    seq_names=['Multiply', 'HorizontalFlip', 'InvertColor', 'PiecewiseAffineTransform', 'RandomTranslate', 'RandomRotate60', 'VerticalFlip', 'GaussianBlur', 'RandomRotate30']
    j=0
    for sq in seqs:
      out_path = os.path.join(output_dir,seq_names[j]+'_'+file)
      out = cv2.VideoWriter(out_path, fourcc, fps, frame_size)
      try:
        video_aug = sq(frames)
        for frame in video_aug:
            if frame.dtype == np.float32 or frame.max() <= 1.0:
              frame = (frame * 255).astype(np.uint8)
            out.write(frame)
        print(f'Augmented video {file} using {seq_names[j]}')
        out.release()
      except Exception as e:
        print(e)
        print(f'Unable to augment: {file} using {seq_names[j]}')
      j+=1

    # Using the code for Perspective Transformation
    video = cv2.VideoCapture(video_path)
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_path = os.path.join(output_dir,'Perspective_Change'+'_'+file)
    out_perspective_update = cv2.VideoWriter(out_path, fourcc, fps, (frame_width, frame_height))
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        uf = update_frame(frame, 10, 20)
        out_perspective_update.write(uf)
    video.release()
    out_perspective_update.release()
    cv2.destroyAllWindows()
    i+=1
